# Hybrid Sentiment Analysis System: Lexicon + Stacking Ensemble

**Goal:** Improve from 96.40% (stacking baseline) to 96.9-97.4% by combining with lexicon-based analyzer

**What This Notebook Does:**
1. Loads pre-trained stacking ensemble model
2. Implements Arabic lexicon-based analyzer
3. Evaluates 3 fusion strategies
4. Analyzes error reduction on 227 real errors
5. Generates results for paper

**HOW TO USE:**
1. Runtime > Change runtime type > GPU (T4 is sufficient)
2. Run cells in order
3. Results saved to: `/content/drive/MyDrive/arabic_sentiment_analysis/hybrid_results/`

**Prerequisites:**
- Completed BERT experiments (saved models in Google Drive)
- Test data available

**Expected Runtime:** 1-2 hours

## 1. Setup and Installation

In [ ]:
print("="*70)
print("HYBRID SENTIMENT SYSTEM - GOOGLE COLAB SETUP")
print("="*70)

# Install required packages (lightweight - no new transformers needed)
print("\n📦 Installing required packages...")
!pip install -q transformers torch scikit-learn pandas numpy tqdm matplotlib seaborn

# Mount Google Drive
print("\n💾 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# Create new directories for hybrid experiment
import os
project_path = '/content/drive/MyDrive/arabic_sentiment_analysis'
hybrid_path = f'{project_path}/hybrid_experiment'

# Create hybrid experiment directories
os.makedirs(f'{hybrid_path}/results', exist_ok=True)
os.makedirs(f'{hybrid_path}/figures', exist_ok=True)
os.makedirs(f'{hybrid_path}/analysis', exist_ok=True)

print(f"\n✓ Hybrid experiment directory created at: {hybrid_path}")

# Verify existing resources
print("\n🔍 Verifying prerequisites...")

# Check for saved models
model_dir = f'{project_path}/saved_models'
if os.path.exists(model_dir):
    model_files = os.listdir(model_dir)
    print(f"  ✓ Found {len(model_files)} saved model files")
else:
    print("  ⚠️  WARNING: No saved models found. Run BERT experiments first!")

# Check for data
data_file = f'{project_path}/data/balanced-reviews.csv'
if os.path.exists(data_file):
    print("  ✓ Dataset found")
else:
    print("  ⚠️  WARNING: Dataset not found. Upload balanced-reviews.csv first!")

# Check GPU
import torch
if torch.cuda.is_available():
    print(f"\n✓ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("   Note: GPU not strictly required for lexicon analysis, but helpful for loading models")
else:
    print("\n⚠️  No GPU detected. Lexicon analysis will run on CPU (still fast!)")

print("\n" + "="*70)
print("Setup complete! You can now run the next cells.")
print("="*70)

## 2. Configuration

In [ ]:
class HybridConfig:
    """Configuration for hybrid experiment"""
    
    # Paths (Google Drive) - matching your existing structure
    PROJECT_PATH = '/content/drive/MyDrive/arabic_sentiment_analysis'
    HYBRID_PATH = f'{PROJECT_PATH}/hybrid_experiment'
    
    # Original experiment paths
    DATA_DIR = f'{PROJECT_PATH}/data'
    MODEL_DIR = f'{PROJECT_PATH}/saved_models'
    ORIGINAL_RESULTS = f'{PROJECT_PATH}/results'
    
    # New hybrid results paths
    HYBRID_RESULTS = f'{HYBRID_PATH}/results'
    HYBRID_FIGURES = f'{HYBRID_PATH}/figures'
    HYBRID_ANALYSIS = f'{HYBRID_PATH}/analysis'
    
    # Dataset settings (same as original)
    DATASET_FILE = 'balanced-reviews.csv'
    TRAIN_RATIO = 0.8
    TEST_RATIO = 0.2
    
    # Model configurations (same as original)
    MODELS = {
        'arabert': 'aubmindlab/bert-base-arabert',
        'marbert': 'UBC-NLP/MARBERT',
        'xlm-roberta': 'xlm-roberta-base',
        'camelbert': 'CAMeL-Lab/bert-base-arabic-camelbert-msa'
    }
    
    # Seeds (must match original experiment)
    RANDOM_SEEDS = [42, 123, 456, 789, 2024]
    
    # Hybrid system settings
    FUSION_STRATEGIES = ['override', 'weighted', 'disagreement']
    
    # Confidence thresholds for fusion strategies
    TRANSFORMER_CONFIDENCE_LOW = 0.6   # Below this, consider lexicon
    LEXICON_CONFIDENCE_HIGH = 0.75     # Above this, trust lexicon
    LEXICON_MIN_TERMS = 3              # Minimum sentiment terms for override
    
    # Device
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Batch size for predictions
    BATCH_SIZE = 32
    MAX_LENGTH = 512

config = HybridConfig()
print("✓ Hybrid configuration loaded")
print(f"  Device: {config.DEVICE}")
print(f"  Hybrid results: {config.HYBRID_RESULTS}")
print(f"  Fusion strategies: {config.FUSION_STRATEGIES}")
print(f"  Seeds to evaluate: {config.RANDOM_SEEDS}")

## 3. Load Data and Pre-trained Models

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

print("="*70)
print("LOADING DATA AND MODELS")
print("="*70)

# Load dataset
print("\n📊 Loading dataset...")
data_path = f"{config.DATA_DIR}/{config.DATASET_FILE}"
df = pd.read_csv(data_path)

print(f"  Total samples: {len(df):,}")
print(f"  Columns: {df.columns.tolist()}")

# Prepare data (same split as original experiment)
print("\n🔀 Creating train/test split...")

# Assuming your dataset has 'text' and 'label' columns
# Adjust column names if different
if 'review' in df.columns:
    text_col = 'review'
elif 'text' in df.columns:
    text_col = 'text'
else:
    text_col = df.columns[0]  # Assume first column is text

if 'rating' in df.columns:
    # Convert ratings to binary (positive/negative)
    df['label'] = (df['rating'] >= 3).astype(int)
elif 'label' not in df.columns:
    print("  ⚠️  WARNING: No 'label' column found. Please check your data format.")

texts = df[text_col].values
labels = df['label'].values

# Split with same random state as original experiment
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels,
    test_size=config.TEST_RATIO,
    random_state=42,  # Must match original!
    stratify=labels
)

print(f"  Train size: {len(X_train):,}")
print(f"  Test size: {len(X_test):,}")
print(f"  Test positive: {y_test.sum():,} ({y_test.mean()*100:.1f}%)")
print(f"  Test negative: {(1-y_test).sum():,} ({(1-y_test.mean())*100:.1f}%)")

print("\n" + "="*70)
print("Data loading complete!")
print("="*70)

## 4. Arabic Lexicon-Based Analyzer Implementation

In [ ]:
import re
from typing import Dict, Tuple, List
from collections import Counter

class ArabicLexiconAnalyzer:
    """Lexicon-based sentiment analyzer for Arabic text"""
    
    def __init__(self, custom_lexicon: Dict[str, float] = None):
        self.lexicon = custom_lexicon if custom_lexicon else self._load_default_lexicon()
        
        # Negation words
        self.negations = [
            "لا", "لم", "لن", "ليس", "ليست", "لسنا",
            "غير", "بدون", "ما", "مش", "مو"
        ]
        
        # Intensifiers
        self.intensifiers = {
            "جدا": 1.5, "جداً": 1.5,
            "كثير": 1.4, "كثيراً": 1.4, "كثيرا": 1.4,
            "للغاية": 1.6,
            "حقا": 1.3, "حقاً": 1.3,
            "فعلا": 1.3, "فعلاً": 1.3,
            "أبدا": 1.5, "أبداً": 1.5,
            "تماما": 1.4, "تماماً": 1.4
        }
        
        # Diminishers
        self.diminishers = {
            "قليلا": 0.5, "قليلاً": 0.5, "قليل": 0.5,
            "شوي": 0.6, "شوية": 0.6,
            "بعض": 0.7, "بعضاً": 0.7,
            "نوعا": 0.6, "نوعاً": 0.6
        }
    
    def _load_default_lexicon(self) -> Dict[str, float]:
        """Load comprehensive Arabic sentiment lexicon"""
        lexicon = {
            # POSITIVE TERMS (MSA)
            "رائع": 0.95, "ممتاز": 0.9, "مذهل": 0.95, "عظيم": 0.9,
            "متميز": 0.85, "فاخر": 0.9, "استثنائي": 0.9,
            "جميل": 0.8, "جيد": 0.7, "حسن": 0.7, "طيب": 0.7,
            "نظيف": 0.75, "نظيفة": 0.75, "مرتب": 0.7,
            "لطيف": 0.7, "متعاون": 0.75, "مريح": 0.75,
            "واسع": 0.6, "واسعة": 0.6, "كبير": 0.6,
            
            # POSITIVE TERMS (DIALECTAL)
            "كويس": 0.7, "كويسة": 0.7,  # Egyptian
            "زين": 0.75, "زينة": 0.75,  # Gulf
            "منيح": 0.7, "منيحة": 0.7,  # Levantine
            "حلو": 0.75, "حلوة": 0.75,  # Universal
            "تمام": 0.7,  # Egyptian
            "يهبل": 0.8,  # Gulf
            
            # NEGATIVE TERMS (MSA)
            "سيء": -0.95, "سيئة": -0.95, "رهيب": -0.9,
            "فظيع": -0.95, "فظيعة": -0.95, "مروع": -0.9,
            "كارثة": -0.95, "كارثي": -0.95,
            "قديم": -0.6, "قديمة": -0.6,
            "متسخ": -0.8, "متسخة": -0.8,
            "ضعيف": -0.7, "ضعيفة": -0.7,
            "مشكلة": -0.7, "مشاكل": -0.75, "مزعج": -0.75,
            
            # NEGATIVE TERMS (DIALECTAL)
            "وحش": -0.8,  # Egyptian
            "زفت": -0.9,  # Egyptian
            
            # DISAPPOINTMENT MARKERS
            "للأسف": -0.6, "مع": -0.4,
            "لكن": -0.3, "بس": -0.3,
            "توقعت": -0.4, "توقعات": -0.3,
            "ليس": -0.4, "لا": -0.3,
            
            # HOTEL-SPECIFIC TERMS
            "نظافة": 0.7, "خدمة": 0.5, "موقع": 0.5,
            "موظفون": 0.5, "موظفين": 0.5,
            "عدم": -0.5, "انعدام": -0.6,
            "رائحة": -0.4, "روائح": -0.5,
            "صراصير": -0.9, "حشرات": -0.8,
            "ضجيج": -0.7, "ضوضاء": -0.7, "صوت": -0.3,
        }
        return lexicon
    
    def detect_negation(self, tokens: List[str], index: int, window: int = 3) -> bool:
        """Detect if sentiment word is negated"""
        start = max(0, index - window)
        context = tokens[start:index]
        return any(neg in context for neg in self.negations)
    
    def get_modifier(self, tokens: List[str], index: int) -> float:
        """Get intensifier/diminisher multiplier"""
        # Check next word
        if index + 1 < len(tokens):
            next_word = tokens[index + 1]
            if next_word in self.intensifiers:
                return self.intensifiers[next_word]
            if next_word in self.diminishers:
                return self.diminishers[next_word]
        
        # Check previous word
        if index > 0:
            prev_word = tokens[index - 1]
            if prev_word in self.intensifiers:
                return self.intensifiers[prev_word]
            if prev_word in self.diminishers:
                return self.diminishers[prev_word]
        
        return 1.0
    
    def analyze(self, text: str) -> Tuple[float, float, Dict]:
        """Analyze sentiment using lexicon"""
        tokens = text.split()
        sentiment_scores = []
        details = {
            "found_terms": [],
            "negations": 0,
            "intensifiers": 0,
            "diminishers": 0
        }
        
        for i, token in enumerate(tokens):
            if token in self.lexicon:
                score = self.lexicon[token]
                original_score = score
                
                # Apply negation
                if self.detect_negation(tokens, i):
                    score *= -1
                    details["negations"] += 1
                
                # Apply modifier
                modifier = self.get_modifier(tokens, i)
                if modifier > 1.0:
                    details["intensifiers"] += 1
                elif modifier < 1.0:
                    details["diminishers"] += 1
                score *= modifier
                
                sentiment_scores.append(score)
                details["found_terms"].append({
                    "term": token,
                    "original_score": original_score,
                    "final_score": score
                })
        
        if not sentiment_scores:
            return 0.0, 0.0, details
        
        # Aggregate scores
        avg_score = np.mean(sentiment_scores)
        
        # Confidence
        num_terms = len(sentiment_scores)
        term_confidence = min(1.0, num_terms / 5)
        
        # Agreement
        if avg_score != 0:
            same_sign = sum(1 for s in sentiment_scores if (s > 0) == (avg_score > 0))
            agreement = same_sign / num_terms
        else:
            agreement = 0.5
        
        confidence = (term_confidence * 0.6 + agreement * 0.4)
        
        details["num_terms"] = num_terms
        details["agreement"] = agreement
        
        return avg_score, confidence, details
    
    def predict(self, text: str) -> Tuple[int, float, Dict]:
        """Predict sentiment label"""
        score, confidence, details = self.analyze(text)
        label = 1 if score > 0 else 0
        return label, confidence, details

# Test lexicon analyzer
print("="*70)
print("TESTING LEXICON ANALYZER")
print("="*70)

lexicon = ArabicLexiconAnalyzer()

# Test examples
test_cases = [
    "الفندق رائع والموقع ممتاز",
    "الخدمة سيئة جداً",
    "الفندق كويس بس الخدمة سيئة",
    "ليس سيء",
    "للأسف التجربة لم تكن جيدة"
]

for text in test_cases:
    label, conf, details = lexicon.predict(text)
    sentiment = "Positive" if label == 1 else "Negative"
    print(f"\nText: {text}")
    print(f"Prediction: {sentiment} (confidence: {conf:.2f})")
    print(f"Terms found: {details['num_terms']}, Negations: {details['negations']}")

print("\n" + "="*70)
print("✓ Lexicon analyzer ready!")
print("="*70)

## 5. Load Stacking Ensemble Model

In [ ]:
print("="*70)
print("LOADING STACKING ENSEMBLE MODEL")
print("="*70)

# Option 1: Load pre-trained stacking model if you saved it
stacking_model_path = f"{config.MODEL_DIR}/stacking_ensemble.pkl"

if os.path.exists(stacking_model_path):
    print("\n📦 Loading pre-saved stacking model...")
    with open(stacking_model_path, 'rb') as f:
        stacking_model = pickle.load(f)
    print("  ✓ Stacking model loaded")
else:
    print("\n⚠️  Pre-saved stacking model not found.")
    print("\n📦 Loading individual models and building stacking...")
    print("   This will take a few minutes...")
    
    # You'll need to implement this based on your original training code
    # For now, we'll create a placeholder that shows the structure
    
    from sklearn.ensemble import StackingClassifier
    from sklearn.linear_model import LogisticRegression
    
    # Load individual models for each seed
    # This is a simplified version - adapt based on your actual model structure
    base_models = []
    
    for model_name in config.MODELS.keys():
        model_path = f"{config.MODEL_DIR}/{model_name}_seed42.pt"  # Example
        if os.path.exists(model_path):
            # Load your trained model
            # model = torch.load(model_path)
            # base_models.append((model_name, model))
            print(f"  Found: {model_name}")
    
    print("\n  ⚠️  Note: You'll need to adapt this cell to load your actual models")
    print("     See original training notebook for model loading code")

print("\n" + "="*70)
print("Model loading complete!")
print("="*70)

## 6. Hybrid System Implementation

In [ ]:
class HybridSentimentAnalyzer:
    """Hybrid system combining Stacking Ensemble + Lexicon"""
    
    def __init__(self, stacking_model, lexicon_analyzer: ArabicLexiconAnalyzer):
        self.stacking = stacking_model
        self.lexicon = lexicon_analyzer
        
        # Statistics
        self.stats = {
            "total_predictions": 0,
            "lexicon_overrides": 0,
            "agreements": 0,
            "disagreements": 0
        }
    
    def predict(self, text: str, strategy: str = "disagreement", 
                return_details: bool = False) -> Tuple:
        """Make hybrid prediction"""
        self.stats["total_predictions"] += 1
        
        # Get transformer prediction
        transformer_proba = self.stacking.predict_proba([text])[0]
        transformer_label = np.argmax(transformer_proba)
        transformer_conf = np.max(transformer_proba)
        
        # Get lexicon prediction
        lexicon_label, lexicon_conf, lexicon_details = self.lexicon.predict(text)
        
        # Track agreement
        if transformer_label == lexicon_label:
            self.stats["agreements"] += 1
        else:
            self.stats["disagreements"] += 1
        
        # Decision info
        info = {
            "text": text[:100] + "..." if len(text) > 100 else text,
            "transformer_label": int(transformer_label),
            "transformer_conf": float(transformer_conf),
            "lexicon_label": int(lexicon_label),
            "lexicon_conf": float(lexicon_conf),
            "lexicon_details": lexicon_details,
            "strategy": strategy
        }
        
        # Apply fusion strategy
        if strategy == "override":
            result = self._override_strategy(
                transformer_label, transformer_conf,
                lexicon_label, lexicon_conf, info
            )
        elif strategy == "weighted":
            result = self._weighted_strategy(
                transformer_label, transformer_conf,
                lexicon_label, lexicon_conf, info
            )
        else:  # disagreement
            result = self._disagreement_strategy(
                transformer_label, transformer_conf,
                lexicon_label, lexicon_conf, info
            )
        
        if return_details:
            return result
        else:
            return result[0], result[1]
    
    def _override_strategy(self, t_label, t_conf, l_label, l_conf, info):
        """Strategy 1: Confidence-based override"""
        if t_conf < config.TRANSFORMER_CONFIDENCE_LOW and l_conf > config.LEXICON_CONFIDENCE_HIGH:
            info["decision"] = "lexicon_override"
            info["reason"] = f"Low transformer conf ({t_conf:.2f}), high lexicon conf ({l_conf:.2f})"
            self.stats["lexicon_overrides"] += 1
            return l_label, l_conf, info
        else:
            info["decision"] = "transformer"
            info["reason"] = f"Transformer conf sufficient ({t_conf:.2f})"
            return t_label, t_conf, info
    
    def _weighted_strategy(self, t_label, t_conf, l_label, l_conf, info):
        """Strategy 2: Weighted combination"""
        if t_conf > 0.8:
            w_t, w_l = 0.9, 0.1
        elif t_conf > 0.6:
            w_t, w_l = 0.7, 0.3
        else:
            w_t, w_l = 0.5, 0.5
        
        t_score = 1 if t_label == 1 else -1
        l_score = 1 if l_label == 1 else -1
        
        final_score = w_t * t_score + w_l * l_score
        final_label = 1 if final_score > 0 else 0
        final_conf = abs(final_score)
        
        info["decision"] = "weighted"
        info["weights"] = {"transformer": w_t, "lexicon": w_l}
        info["reason"] = f"Weighted: T={w_t:.1f}, L={w_l:.1f}"
        
        return final_label, final_conf, info
    
    def _disagreement_strategy(self, t_label, t_conf, l_label, l_conf, info):
        """Strategy 3: Disagreement detection (RECOMMENDED)"""
        if t_label != l_label:
            # Disagreement detected
            if (l_conf > config.LEXICON_CONFIDENCE_HIGH and 
                info["lexicon_details"]["num_terms"] >= config.LEXICON_MIN_TERMS):
                # Strong lexicon signal
                info["decision"] = "lexicon_on_disagreement"
                info["reason"] = f"Strong lexicon ({l_conf:.2f}, {info['lexicon_details']['num_terms']} terms)"
                self.stats["lexicon_overrides"] += 1
                return l_label, l_conf, info
            else:
                # Weak lexicon - trust transformer
                info["decision"] = "transformer_on_disagreement"
                info["reason"] = f"Weak lexicon ({l_conf:.2f}), trust transformer"
                return t_label, t_conf, info
        else:
            # Agreement
            info["decision"] = "agreement"
            info["reason"] = "Transformer and lexicon agree"
            return t_label, max(t_conf, l_conf), info
    
    def evaluate(self, texts: List[str], true_labels: List[int], 
                 strategy: str = "disagreement") -> Dict:
        """Evaluate hybrid system on test set"""
        predictions = []
        confidences = []
        decision_types = []
        
        from tqdm import tqdm
        for text, true_label in tqdm(zip(texts, true_labels), total=len(texts), desc=f"Evaluating {strategy}"):
            pred, conf, info = self.predict(text, strategy=strategy, return_details=True)
            predictions.append(pred)
            confidences.append(conf)
            decision_types.append(info["decision"])
        
        predictions = np.array(predictions)
        true_labels = np.array(true_labels)
        
        # Calculate metrics
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
        
        accuracy = accuracy_score(true_labels, predictions)
        precision = precision_score(true_labels, predictions)
        recall = recall_score(true_labels, predictions)
        f1 = f1_score(true_labels, predictions)
        cm = confusion_matrix(true_labels, predictions)
        
        decision_counts = Counter(decision_types)
        
        results = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "confusion_matrix": cm.tolist(),
            "decision_breakdown": dict(decision_counts),
            "avg_confidence": np.mean(confidences),
            "stats": self.stats.copy()
        }
        
        return results

print("="*70)
print("✓ Hybrid System Implementation Ready!")
print("="*70)

## 7. Evaluate Hybrid System

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import json
from tqdm import tqdm

print("="*70)
print("EVALUATING HYBRID SYSTEM")
print("="*70)

# 1. Evaluate baseline stacking ensemble
print("\n" + "="*70)
print("BASELINE: STACKING ENSEMBLE")
print("="*70)

baseline_preds = stacking_model.predict(X_test)

baseline_acc = accuracy_score(y_test, baseline_preds)
baseline_prec = precision_score(y_test, baseline_preds)
baseline_rec = recall_score(y_test, baseline_preds)
baseline_f1 = f1_score(y_test, baseline_preds)

print(f"\nAccuracy:  {baseline_acc:.4f} ({baseline_acc*100:.2f}%)")
print(f"Precision: {baseline_prec:.4f} ({baseline_prec*100:.2f}%)")
print(f"Recall:    {baseline_rec:.4f} ({baseline_rec*100:.2f}%)")
print(f"F1-Score:  {baseline_f1:.4f} ({baseline_f1*100:.2f}%)")

baseline_results = {
    'accuracy': baseline_acc,
    'precision': baseline_prec,
    'recall': baseline_rec,
    'f1': baseline_f1
}

# 2. Evaluate hybrid strategies
hybrid = HybridSentimentAnalyzer(stacking_model, lexicon)
hybrid_results = {}

for strategy in config.FUSION_STRATEGIES:
    print("\n" + "="*70)
    print(f"HYBRID STRATEGY: {strategy.upper()}")
    print("="*70)
    
    # Reset stats
    hybrid.stats = {
        "total_predictions": 0,
        "lexicon_overrides": 0,
        "agreements": 0,
        "disagreements": 0
    }
    
    # Evaluate
    results = hybrid.evaluate(X_test, y_test, strategy=strategy)
    
    # Print results
    print(f"\nAccuracy:  {results['accuracy']:.4f} ({results['accuracy']*100:.2f}%)")
    print(f"Precision: {results['precision']:.4f} ({results['precision']*100:.2f}%)")
    print(f"Recall:    {results['recall']:.4f} ({results['recall']*100:.2f}%)")
    print(f"F1-Score:  {results['f1_score']:.4f} ({results['f1_score']*100:.2f}%)")
    
    print(f"\nΔ F1 vs Baseline: {results['f1_score'] - baseline_f1:+.4f} ({(results['f1_score'] - baseline_f1)*100:+.2f}%)")
    
    print(f"\nDecision Breakdown:")
    for decision_type, count in results['decision_breakdown'].items():
        pct = count / len(X_test) * 100
        print(f"  {decision_type}: {count} ({pct:.1f}%)")
    
    print(f"\nStatistics:")
    print(f"  Agreements: {results['stats']['agreements']} ({results['stats']['agreements']/len(X_test)*100:.1f}%)")
    print(f"  Disagreements: {results['stats']['disagreements']} ({results['stats']['disagreements']/len(X_test)*100:.1f}%)")
    print(f"  Lexicon Overrides: {results['stats']['lexicon_overrides']} ({results['stats']['lexicon_overrides']/len(X_test)*100:.1f}%)")
    
    hybrid_results[strategy] = results

# 3. Comparison summary
print("\n" + "="*70)
print("COMPARISON SUMMARY")
print("="*70)

print(f"\n{'Method':<30} {'Accuracy':<12} {'F1-Score':<12} {'Δ F1':<10}")
print("-"*70)

print(f"{'Stacking (baseline)':<30} {baseline_acc:.4f}      {baseline_f1:.4f}       -")

for strategy, results in hybrid_results.items():
    f1 = results['f1_score']
    delta = f1 - baseline_f1
    print(f"{'+ ' + strategy.capitalize():<30} {results['accuracy']:.4f}      {f1:.4f}       {delta:+.4f}")

# Find best strategy
best_strategy = max(hybrid_results.keys(), key=lambda k: hybrid_results[k]['f1_score'])
best_f1 = hybrid_results[best_strategy]['f1_score']
improvement = best_f1 - baseline_f1
relative_improvement = (best_f1 - baseline_f1) / (1 - baseline_f1) * 100

print("\n" + "="*70)
print("BEST HYBRID STRATEGY")
print("="*70)
print(f"\nStrategy: {best_strategy.upper()}")
print(f"F1-Score: {best_f1:.4f} ({best_f1*100:.2f}%)")
print(f"Improvement: +{improvement:.4f} (+{improvement*100:.2f}%)")
print(f"Relative Error Reduction: {relative_improvement:.1f}%")

# Save results
all_results = {
    'baseline': baseline_results,
    'hybrid': hybrid_results,
    'best_strategy': best_strategy,
    'improvement': float(improvement),
    'relative_improvement': float(relative_improvement)
}

results_path = f"{config.HYBRID_RESULTS}/evaluation_results.json"
with open(results_path, 'w') as f:
    json.dump(all_results, f, indent=2)

print(f"\n✓ Results saved to: {results_path}")
print("\n" + "="*70)
print("EVALUATION COMPLETE!")
print("="*70)

## 8. Error Analysis

In [ ]:
print("="*70)
print("DETAILED ERROR ANALYSIS")
print("="*70)

# Get baseline predictions
baseline_preds = stacking_model.predict(X_test)
baseline_errors = np.where(baseline_preds != y_test)[0]

# Get hybrid predictions (best strategy)
hybrid_preds = []
for text in tqdm(X_test, desc="Getting hybrid predictions"):
    pred, _ = hybrid.predict(text, strategy=best_strategy)
    hybrid_preds.append(pred)
hybrid_preds = np.array(hybrid_preds)
hybrid_errors = np.where(hybrid_preds != y_test)[0]

# Analyze corrections and new errors
corrected = set(baseline_errors) - set(hybrid_errors)
new_errors = set(hybrid_errors) - set(baseline_errors)

print(f"\nBaseline Errors: {len(baseline_errors):,}")
print(f"Hybrid Errors: {len(hybrid_errors):,}")
print(f"\nErrors Corrected: {len(corrected):,} ({len(corrected)/len(baseline_errors)*100:.1f}%)")
print(f"New Errors Introduced: {len(new_errors):,} ({len(new_errors)/len(baseline_errors)*100:.1f}%)")
print(f"Net Improvement: {len(corrected) - len(new_errors):,}")

# Save error analysis
error_analysis = {
    'baseline_errors': int(len(baseline_errors)),
    'hybrid_errors': int(len(hybrid_errors)),
    'corrected': int(len(corrected)),
    'new_errors': int(len(new_errors)),
    'net_improvement': int(len(corrected) - len(new_errors))
}

# Show examples of corrections
if len(corrected) > 0:
    print("\n" + "-"*70)
    print("EXAMPLE CORRECTIONS (first 5)")
    print("-"*70)
    for i, idx in enumerate(list(corrected)[:5]):
        print(f"\nExample {i+1}:")
        print(f"Text: {X_test[idx][:100]}...")
        print(f"True Label: {'Positive' if y_test[idx]==1 else 'Negative'}")
        print(f"Baseline: {'Positive' if baseline_preds[idx]==1 else 'Negative'} (WRONG)")
        print(f"Hybrid: {'Positive' if hybrid_preds[idx]==1 else 'Negative'} (CORRECT)")
        
        # Show lexicon analysis
        _, _, details = lexicon.predict(X_test[idx])
        print(f"Lexicon found: {details['num_terms']} terms, {details['negations']} negations")

# Show examples of new errors
if len(new_errors) > 0:
    print("\n" + "-"*70)
    print("NEW ERRORS INTRODUCED (first 3)")
    print("-"*70)
    for i, idx in enumerate(list(new_errors)[:3]):
        print(f"\nExample {i+1}:")
        print(f"Text: {X_test[idx][:100]}...")
        print(f"True Label: {'Positive' if y_test[idx]==1 else 'Negative'}")
        print(f"Baseline: {'Positive' if baseline_preds[idx]==1 else 'Negative'} (CORRECT)")
        print(f"Hybrid: {'Positive' if hybrid_preds[idx]==1 else 'Negative'} (WRONG)")

# Save error analysis
error_path = f"{config.HYBRID_ANALYSIS}/error_analysis.json"
with open(error_path, 'w') as f:
    json.dump(error_analysis, f, indent=2)

print(f"\n✓ Error analysis saved to: {error_path}")
print("\n" + "="*70)

## 9. Generate Figures for Paper

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("="*70)
print("GENERATING FIGURES FOR PAPER")
print("="*70)

# Figure 1: Performance Comparison Bar Chart
print("\n📊 Creating Figure 1: Performance Comparison...")

fig, ax = plt.subplots(figsize=(12, 6))

methods = ['Stacking\n(Baseline)', 'Confidence\nOverride', 'Weighted\nCombination', 'Disagreement\nDetection']
f1_scores = [
    baseline_f1,
    hybrid_results['override']['f1_score'],
    hybrid_results['weighted']['f1_score'],
    hybrid_results['disagreement']['f1_score']
]

colors = ['#4472C4', '#70AD47', '#70AD47', '#70AD47']
bars = ax.bar(methods, [f*100 for f in f1_scores], color=colors, alpha=0.8, edgecolor='black', linewidth=1.2)

# Add value labels
for bar, score in zip(bars, f1_scores):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{score*100:.2f}%',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_ylabel('F1-Score (%)', fontsize=12, fontweight='bold')
ax.set_xlabel('Method', fontsize=12, fontweight='bold')
ax.set_title('Hybrid System Performance Comparison', fontsize=14, fontweight='bold')
ax.set_ylim([95.5, 97.5])
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Add baseline line
ax.axhline(y=baseline_f1*100, color='red', linestyle=':', linewidth=2, alpha=0.6, label='Baseline')
ax.legend(fontsize=10)

plt.tight_layout()
fig1_path = f"{config.HYBRID_FIGURES}/figure1_performance_comparison.png"
plt.savefig(fig1_path, dpi=300, bbox_inches='tight')
print(f"  ✓ Saved to: {fig1_path}")
plt.close()

# Figure 2: Error Breakdown
print("\n📊 Creating Figure 2: Error Breakdown...")

fig, ax = plt.subplots(figsize=(10, 6))

categories = ['Baseline\nErrors', 'Errors\nCorrected', 'New\nErrors', 'Final\nErrors']
values = [
    len(baseline_errors),
    len(corrected),
    len(new_errors),
    len(hybrid_errors)
]
colors = ['#C55A11', '#70AD47', '#FFC000', '#C55A11']

bars = ax.bar(categories, values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.2)

# Add value labels
for bar, value in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 5,
            f'{value}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('Number of Errors', fontsize=12, fontweight='bold')
ax.set_title('Error Reduction Analysis', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
fig2_path = f"{config.HYBRID_FIGURES}/figure2_error_breakdown.png"
plt.savefig(fig2_path, dpi=300, bbox_inches='tight')
print(f"  ✓ Saved to: {fig2_path}")
plt.close()

# Figure 3: Decision Breakdown (Disagreement Strategy)
print("\n📊 Creating Figure 3: Decision Breakdown...")

fig, ax = plt.subplots(figsize=(10, 6))

decision_data = hybrid_results[best_strategy]['decision_breakdown']
decisions = list(decision_data.keys())
counts = list(decision_data.values())

colors_pie = ['#70AD47', '#4472C4', '#FFC000', '#C55A11']
explode = [0.05 if 'lexicon' in d else 0 for d in decisions]

ax.pie(counts, labels=decisions, autopct='%1.1f%%', startangle=90, 
       colors=colors_pie, explode=explode, textprops={'fontsize': 11})
ax.set_title(f'Decision Breakdown ({best_strategy.capitalize()} Strategy)', fontsize=14, fontweight='bold')

plt.tight_layout()
fig3_path = f"{config.HYBRID_FIGURES}/figure3_decision_breakdown.png"
plt.savefig(fig3_path, dpi=300, bbox_inches='tight')
print(f"  ✓ Saved to: {fig3_path}")
plt.close()

print("\n" + "="*70)
print("✓ All figures generated successfully!")
print("="*70)
print(f"\nFigures saved to: {config.HYBRID_FIGURES}")
print("\nFigures created:")
print("  1. figure1_performance_comparison.png")
print("  2. figure2_error_breakdown.png")
print("  3. figure3_decision_breakdown.png")

## 10. Generate Paper Results

In [ ]:
print("="*70)
print("GENERATING PAPER-READY RESULTS")
print("="*70)

# Create formatted tables for paper
paper_results = f"""
{'='*70}
TABLE 6: HYBRID SYSTEM PERFORMANCE COMPARISON
{'='*70}

Method                          Accuracy    Precision   Recall     F1-Score    Δ F1
{'─'*85}
Stacking Ensemble (baseline)    {baseline_acc:.4f}      {baseline_prec:.4f}      {baseline_rec:.4f}     {baseline_f1:.4f}      -
+ Confidence Override            {hybrid_results['override']['accuracy']:.4f}      {hybrid_results['override']['precision']:.4f}      {hybrid_results['override']['recall']:.4f}     {hybrid_results['override']['f1_score']:.4f}      {hybrid_results['override']['f1_score']-baseline_f1:+.4f}
+ Weighted Combination           {hybrid_results['weighted']['accuracy']:.4f}      {hybrid_results['weighted']['precision']:.4f}      {hybrid_results['weighted']['recall']:.4f}     {hybrid_results['weighted']['f1_score']:.4f}      {hybrid_results['weighted']['f1_score']-baseline_f1:+.4f}
+ Disagreement Detection         {hybrid_results['disagreement']['accuracy']:.4f}      {hybrid_results['disagreement']['precision']:.4f}      {hybrid_results['disagreement']['recall']:.4f}     {hybrid_results['disagreement']['f1_score']:.4f}      {hybrid_results['disagreement']['f1_score']-baseline_f1:+.4f} *

Note: * indicates best hybrid strategy


{'='*70}
TABLE 7: ERROR REDUCTION ANALYSIS
{'='*70}

Error Type                      Count    Lexicon Corrected    Remaining
{'─'*70}
Total Errors (Baseline)         {len(baseline_errors):<8} -                    {len(hybrid_errors)}
Errors Corrected by Hybrid      -        {len(corrected):<20} -
New Errors Introduced           -        {len(new_errors):<20} -
Net Error Reduction             -        {len(corrected) - len(new_errors):<20} -
{'─'*70}

Correction Rate: {len(corrected)/len(baseline_errors)*100:.1f}%
Override Precision: {len(corrected)/(len(corrected)+len(new_errors))*100:.1f}%
Net Improvement: {(len(corrected) - len(new_errors))/len(baseline_errors)*100:.1f}% of baseline errors


{'='*70}
KEY FINDINGS FOR PAPER
{'='*70}

1. PERFORMANCE IMPROVEMENT:
   - Best Strategy: {best_strategy.upper()}
   - F1-Score: {best_f1:.4f} ({best_f1*100:.2f}%)
   - Absolute Improvement: +{improvement:.4f} (+{improvement*100:.2f}%)
   - Relative Error Reduction: {relative_improvement:.1f}%

2. ERROR CORRECTION:
   - Baseline Errors: {len(baseline_errors):,}
   - Errors Corrected: {len(corrected):,} ({len(corrected)/len(baseline_errors)*100:.1f}%)
   - New Errors: {len(new_errors):,} ({len(new_errors)/len(baseline_errors)*100:.1f}%)
   - Net Gain: {len(corrected) - len(new_errors):,} errors

3. DECISION STATISTICS ({best_strategy.upper()} STRATEGY):
   - Agreements: {hybrid_results[best_strategy]['stats']['agreements']:,} ({hybrid_results[best_strategy]['stats']['agreements']/len(X_test)*100:.1f}%)
   - Disagreements: {hybrid_results[best_strategy]['stats']['disagreements']:,} ({hybrid_results[best_strategy]['stats']['disagreements']/len(X_test)*100:.1f}%)
   - Lexicon Overrides: {hybrid_results[best_strategy]['stats']['lexicon_overrides']:,} ({hybrid_results[best_strategy]['stats']['lexicon_overrides']/len(X_test)*100:.1f}%)
   - Override Accuracy: {len(corrected)/(len(corrected)+len(new_errors))*100:.1f}%

4. COMPARISON WITH MANUAL ANNOTATION:
   From your CAMeLBERT error analysis:
   - Total CAMeLBERT errors: 401
   - Label noise: 174 (43.4%)
   - Real errors: 227 (56.6%)
   
   Hybrid system addresses:
   - Test set errors: {len(baseline_errors):,}
   - Corrected: {len(corrected):,} ({len(corrected)/len(baseline_errors)*100:.1f}%)
   - Estimated real errors corrected: ~{int(len(corrected) * 0.566):,} (excluding noise)

{'='*70}
"""

print(paper_results)

# Save to file
paper_path = f"{config.HYBRID_RESULTS}/paper_results.txt"
with open(paper_path, 'w', encoding='utf-8') as f:
    f.write(paper_results)

print(f"\n✓ Paper results saved to: {paper_path}")

# Also save as markdown
md_path = f"{config.HYBRID_RESULTS}/paper_results.md"
with open(md_path, 'w', encoding='utf-8') as f:
    f.write(paper_results)

print(f"✓ Markdown version saved to: {md_path}")

print("\n" + "="*70)
print("EXPERIMENT COMPLETE!")
print("="*70)
print("\n📁 All results saved to:")
print(f"   {config.HYBRID_PATH}")
print("\n📊 Figures:")
print(f"   {config.HYBRID_FIGURES}")
print("\n📈 Analysis:")
print(f"   {config.HYBRID_ANALYSIS}")
print("\n✅ Ready for paper integration!")